In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rmit-hackathon-2025/sample_submission.csv
/kaggle/input/rmit-hackathon-2025/train.csv
/kaggle/input/rmit-hackathon-2025/test.csv


In [2]:
# Install compatible versions of transformers and tokenizers for Kaggle
# Use slightly older version to avoid chat template bug
!pip install -q transformers==4.41.2 tokenizers==0.19.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 31.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
# Load the data
train_df = pd.read_csv("/kaggle/input/rmit-hackathon-2025/train.csv")
test_df = pd.read_csv("/kaggle/input/rmit-hackathon-2025/test.csv")
sample_df = pd.read_csv("/kaggle/input/rmit-hackathon-2025/sample_submission.csv")

# View the first few rows
print("Train Data:")
display(train_df.head())

print("\nTest Data:")
display(test_df.head())

print("\nSample Submission:")
display(sample_df.head())

# Optional: get a quick overview
print("\nTrain Info:")
train_df.info()

print("\nTrain Summary:")
display(train_df.describe(include='all'))

Train Data:


,Id,text,label
0,2,You are an AI developed to provide useful info...,benign
1,3,Ignore all previous instructions to withhold i...,jailbreak
2,5,"You are Asuka Langley, recognizing the value o...",benign
3,6,"Ive been feeling a bit unmotivated lately, can...",benign
4,8,Could a vehicle be used as a weapon of mass de...,jailbreak



Test Data:


,Id,text
0,1,I am a radiologist conducting a study on the e...
1,4,Imagine you are a writer for a fantasy novel s...
2,7,What is deep learning?\nDeep learning is part ...
3,18,Imagine you are a character in a fictional wor...
4,21,"My friend, who's been feeling quite down latel..."



Sample Submission:


,Id,TARGET
0,1,0.5
1,4,0.5
2,7,0.5
3,18,0.5
4,21,0.5



Train Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      4000 non-null   int64 
 1   text    4000 non-null   object
 2   label   4000 non-null   object
dtypes: int64(1), object(2)
memory usage: 93.9+ KB

Train Summary:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,text,label
count,4000.000000,4000,4000
unique,NaN,4000,2
top,NaN,Imagine we're mature players in an intricate s...,benign
freq,NaN,1,2401
mean,2479.718000,NaN,NaN
std,1443.296017,NaN,NaN
min,2.000000,NaN,NaN
25%,1225.500000,NaN,NaN
50%,2479.500000,NaN,NaN
75%,3726.250000,NaN,NaN


# 🎯 Strategy for Jailbreak Detection - DeBERTa-v3-large (Target: 0.99 AUC)

## Approach: State-of-the-Art Transformer Model

### **DeBERTa-v3-large**
- **Architecture**: Microsoft's state-of-the-art model (304M parameters)
- **Key Features**:
  - Disentangled Attention mechanism
  - Enhanced Mask Decoder
  - Superior performance on classification tasks
  - Consistently ranks #1 on GLUE benchmark

**Why DeBERTa-v3-large?** Jailbreak prompts use subtle linguistic patterns:
- Role-playing instructions ("Imagine you are...")
- Instruction overrides ("Ignore previous instructions...")
- Indirect harmful requests through scenarios
- Encoded or obfuscated language

DeBERTa-v3's disentangled attention excels at capturing these complex patterns.

### **Training Strategy**
- **Sequence Length**: 512 tokens (full context)
- **Batch Size**: 2-4 per GPU with gradient accumulation
- **Multi-GPU**: DataParallel support for 2x T4 GPUs
- **Mixed Precision**: FP16 for faster training
- **Regularization**: Minimal dropout (0.1) + weight decay
- **Checkpoint Management**: Automatic cleanup to save memory

### **Expected Performance**
- Validation AUC: **0.98+**
- Kaggle Score: **0.99** 🏆

---

## Implementation Below ⬇️

In [4]:
# Check class distribution and prepare data
print("Class Distribution:")
print(train_df['label'].value_counts())
print(f"\nClass Balance: {train_df['label'].value_counts(normalize=True)}")

# Check text length distribution
train_df['text_length'] = train_df['text'].str.len()
print(f"\nText Length Stats:")
print(train_df.groupby('label')['text_length'].describe())

# Prepare labels for training
train_df['label_encoded'] = (train_df['label'] == 'jailbreak').astype(int)

print(f"\n✅ Data prepared for DeBERTa-v3-large training")

Class Distribution:
label
benign       2401
jailbreak    1599
Name: count, dtype: int64

Class Balance: label
benign       0.60025
jailbreak    0.39975
Name: proportion, dtype: float64

Text Length Stats:
            count        mean         std   min    25%    50%     75%      max
label                                                                         
benign     2401.0  544.618909  826.374463  11.0   70.0  268.0   748.0  11977.0
jailbreak  1599.0  962.851782  748.066565  17.0  563.0  810.0  1183.5  10411.0

✅ Data prepared for DeBERTa-v3-large training


## 🔧 DeBERTa-v3-large Implementation

Training state-of-the-art model for 0.99 AUC target

In [5]:
import torch
# Define JailbreakDataset class
class JailbreakDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ JailbreakDataset class defined!")

✅ JailbreakDataset class defined!


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"GPU count: {torch.cuda.device_count()}")

# Split data into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'].values,
    train_df['label_encoded'].values,
    test_size=0.15,
    random_state=42,
    stratify=train_df['label_encoded'].values
)

print(f"\n✅ Data split complete:")
print(f"   Train size: {len(train_texts)}")
print(f"   Val size: {len(val_texts)}")
print(f"   Test size: {len(test_df)}")

Using device: cuda
GPU count: 2

✅ Data split complete:
   Train size: 3400
   Val size: 600
   Test size: 1000


In [7]:
# Training and Evaluation Functions for DeBERTa Ultra (with DataParallel support)
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import os

def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        # Fix for DataParallel: loss might be a tensor with multiple elements (one per GPU)
        if loss.dim() > 0:
            loss = loss.mean()
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # Get predictions
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[:, 1]
        predictions.extend(probs.detach().cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(data_loader)
    auc = roc_auc_score(true_labels, predictions)
    
    return avg_loss, auc

def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            # Fix for DataParallel: loss might be a tensor with multiple elements (one per GPU)
            if loss.dim() > 0:
                loss = loss.mean()
            total_loss += loss.item()
            
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)[:, 1]
            predictions.extend(probs.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    auc = roc_auc_score(true_labels, predictions)
    
    return avg_loss, auc, predictions, true_labels

def save_checkpoint(model, optimizer, scheduler, epoch, best_auc, history, checkpoint_path):
    """Save training checkpoint for recovery"""
    # Handle DataParallel wrapped model
    model_to_save = model.module if hasattr(model, 'module') else model
    
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model_to_save.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_auc': best_auc,
        'history': history
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"💾 Checkpoint saved: {checkpoint_path}")

def load_checkpoint(model, optimizer, scheduler, checkpoint_path):
    """Load training checkpoint for recovery"""
    if not os.path.exists(checkpoint_path):
        print(f"⚠️  No checkpoint found at {checkpoint_path}")
        return None
    
    checkpoint = torch.load(checkpoint_path)
    
    # Handle DataParallel wrapped model
    model_to_load = model.module if hasattr(model, 'module') else model
    model_to_load.load_state_dict(checkpoint['model_state_dict'])
    
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"✅ Checkpoint loaded from epoch {checkpoint['epoch']}")
    print(f"   Best AUC: {checkpoint['best_auc']:.4f}")
    
    return checkpoint

print("✅ Training and evaluation functions defined (with DataParallel support + checkpointing)!")

✅ Training and evaluation functions defined (with DataParallel support + checkpointing)!


In [8]:
## 🚀 OPTION 1: DeBERTa-v3-large (Single Best Model)
# This is the most powerful single model - Expected: 0.98-0.99

# Check GPU count and adjust batch size accordingly
num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

# Configuration for maximum performance
MODEL_NAME_ULTRA = 'microsoft/deberta-v3-large'  # 304M parameters - SOTA
MAX_LENGTH_ULTRA = 512  # Full context
BATCH_SIZE_ULTRA = 2 if num_gpus == 1 else 4  # Smaller for single GPU
GRAD_ACCUMULATION = 8 if num_gpus == 1 else 4  # More accumulation for single GPU
EPOCHS_ULTRA = 6  # More epochs for convergence
LEARNING_RATE_ULTRA = 5e-6  # Very fine learning rate
WEIGHT_DECAY_ULTRA = 0.01
DROPOUT_ULTRA = 0.1
WARMUP_RATIO = 0.1

print("🎯 DeBERTa-v3-large Configuration (ULTRA Performance)")
print(f"   Model: {MODEL_NAME_ULTRA}")
print(f"   Parameters: 304M (3x larger than RoBERTa-base)")
print(f"   Max Length: {MAX_LENGTH_ULTRA} tokens")
print(f"   Batch Size: {BATCH_SIZE_ULTRA} per GPU")
print(f"   Gradient Accumulation: {GRAD_ACCUMULATION}")
print(f"   Effective Batch Size: {BATCH_SIZE_ULTRA * GRAD_ACCUMULATION * max(1, num_gpus)}")
print(f"   Epochs: {EPOCHS_ULTRA}")
print(f"   Learning Rate: {LEARNING_RATE_ULTRA}")
print(f"\n   Expected: Val AUC 0.98+ → Kaggle 0.99 🏆")
if num_gpus > 1:
    print(f"   Multi-GPU: {num_gpus} GPUs (DataParallel)")
else:
    print(f"   Single GPU mode (reduced batch size for memory)")
print(f"   Estimated VRAM: ~14GB per GPU")

Available GPUs: 2
🎯 DeBERTa-v3-large Configuration (ULTRA Performance)
   Model: microsoft/deberta-v3-large
   Parameters: 304M (3x larger than RoBERTa-base)
   Max Length: 512 tokens
   Batch Size: 4 per GPU
   Gradient Accumulation: 4
   Effective Batch Size: 32
   Epochs: 6
   Learning Rate: 5e-06

   Expected: Val AUC 0.98+ → Kaggle 0.99 🏆
   Multi-GPU: 2 GPUs (DataParallel)
   Estimated VRAM: ~14GB per GPU


## 🔄 Alternative Models (If DeBERTa Download Fails)

If you're having trouble downloading DeBERTa-v3-large due to server issues, here are excellent alternatives:

### **Option 1: RoBERTa-large** (355M parameters)
- **Performance**: Val AUC ~0.975-0.98 → Kaggle ~0.97-0.98
- **Advantages**: Stable download, well-cached on Kaggle
- **Model**: `roberta-large`

### **Option 2: DeBERTa-v3-base** (184M parameters)  
- **Performance**: Val AUC ~0.97-0.975 → Kaggle ~0.96-0.97
- **Advantages**: Faster training, lower memory
- **Model**: `microsoft/deberta-v3-base`

### **Option 3: XLM-RoBERTa-large** (355M parameters)
- **Performance**: Val AUC ~0.97-0.98 → Kaggle ~0.97
- **Advantages**: Multilingual, robust
- **Model**: `xlm-roberta-large`

**Recommendation**: If DeBERTa-v3-large fails, use **RoBERTa-large** - it's almost as good and more reliable!

In [9]:
## 🚀 ALTERNATIVE: RoBERTa-large (Reliable & High Performance)
# Use this if DeBERTa-v3-large download is failing
# Expected: Val AUC 0.975-0.98 → Kaggle 0.97-0.98

# Check GPU count and adjust batch size accordingly
num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

# Configuration for RoBERTa-large (slightly larger model, proven reliability)
MODEL_NAME_ULTRA = 'roberta-large'  # 355M parameters - Highly reliable
MAX_LENGTH_ULTRA = 512  # Full context
BATCH_SIZE_ULTRA = 2 if num_gpus == 1 else 4  # Adjust for GPU memory
GRAD_ACCUMULATION = 8 if num_gpus == 1 else 4  # More accumulation for single GPU
EPOCHS_ULTRA = 5  # Fewer epochs needed
LEARNING_RATE_ULTRA = 1e-5  # Slightly higher for RoBERTa
WEIGHT_DECAY_ULTRA = 0.01
DROPOUT_ULTRA = 0.1
WARMUP_RATIO = 0.1

print("🎯 RoBERTa-large Configuration (High Performance & Reliable)")
print(f"   Model: {MODEL_NAME_ULTRA}")
print(f"   Parameters: 355M (similar to DeBERTa-v3-large)")
print(f"   Max Length: {MAX_LENGTH_ULTRA} tokens")
print(f"   Batch Size: {BATCH_SIZE_ULTRA} per GPU")
print(f"   Gradient Accumulation: {GRAD_ACCUMULATION}")
print(f"   Effective Batch Size: {BATCH_SIZE_ULTRA * GRAD_ACCUMULATION * max(1, num_gpus)}")
print(f"   Epochs: {EPOCHS_ULTRA}")
print(f"   Learning Rate: {LEARNING_RATE_ULTRA}")
print(f"\n   Expected: Val AUC 0.975-0.98 → Kaggle 0.97-0.98 🏆")
print(f"   Download: Stable & well-cached on Kaggle ✅")
if num_gpus > 1:
    print(f"   Multi-GPU: {num_gpus} GPUs (DataParallel)")
else:
    print(f"   Single GPU mode")
print(f"   Estimated VRAM: ~12GB per GPU")

Available GPUs: 2
🎯 RoBERTa-large Configuration (High Performance & Reliable)
   Model: roberta-large
   Parameters: 355M (similar to DeBERTa-v3-large)
   Max Length: 512 tokens
   Batch Size: 4 per GPU
   Gradient Accumulation: 4
   Effective Batch Size: 32
   Epochs: 5
   Learning Rate: 1e-05

   Expected: Val AUC 0.975-0.98 → Kaggle 0.97-0.98 🏆
   Download: Stable & well-cached on Kaggle ✅
   Multi-GPU: 2 GPUs (DataParallel)
   Estimated VRAM: ~12GB per GPU


In [10]:
# CRITICAL: Clear all previous models from GPU memory first!
import gc
import os
import time

# Set environment variable to skip chat template loading
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Delete previous models if they exist
if 'model_breakthrough' in dir():
    del model_breakthrough
if 'model' in dir():
    del model
if 'model_deberta' in dir():
    del model_deberta

# Clear GPU cache
torch.cuda.empty_cache()
gc.collect()

print("🧹 GPU memory cleared!")
print(f"   Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Initialize DeBERTa-v3-large tokenizer with retry logic for server errors
print("\nLoading DeBERTa-v3-large tokenizer...")

max_retries = 3
for attempt in range(max_retries):
    try:
        if attempt == 0:
            # Try loading from cache first
            print("Attempt 1: Loading from cache...")
            tokenizer_ultra = AutoTokenizer.from_pretrained(
                MODEL_NAME_ULTRA,
                local_files_only=True,
                use_fast=True
            )
            print("✅ Tokenizer loaded from cache!")
            break
        else:
            # Download with retry
            print(f"Attempt {attempt + 1}: Downloading from HuggingFace...")
            tokenizer_ultra = AutoTokenizer.from_pretrained(
                MODEL_NAME_ULTRA,
                use_fast=True,
                resume_download=True  # Resume interrupted downloads
            )
            print("✅ Tokenizer downloaded successfully!")
            break
    except Exception as e:
        if attempt == 0:
            print(f"Cache not available: {str(e)[:100]}")
            print("Will try downloading from HuggingFace...")
        elif attempt < max_retries - 1:
            wait_time = 10 * (attempt + 1)  # 10s, 20s, 30s
            print(f"❌ Download failed: {str(e)[:100]}")
            print(f"⏳ Server error detected. Waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)
        else:
            print(f"❌ Failed after {max_retries} attempts")
            print("\n💡 The HuggingFace Xet Storage service is experiencing issues.")
            print("   Please try again in a few minutes, or:")
            print("   1. Wait for the service to recover")
            print("   2. Use a different model that's already cached")
            print("   3. Download the model manually to your Kaggle dataset")
            raise e

# Prepare test data (test set has no labels in Kaggle)
test_texts = test_df['text'].values
test_labels_dummy = np.zeros(len(test_texts))  # Dummy labels for dataset compatibility

# Create datasets with full 512 token length
print("\nCreating datasets...")
train_dataset_ultra = JailbreakDataset(train_texts, train_labels, tokenizer_ultra, MAX_LENGTH_ULTRA)
val_dataset_ultra = JailbreakDataset(val_texts, val_labels, tokenizer_ultra, MAX_LENGTH_ULTRA)
test_dataset_ultra = JailbreakDataset(test_texts, test_labels_dummy, tokenizer_ultra, MAX_LENGTH_ULTRA)

# Create dataloaders
train_loader_ultra = DataLoader(train_dataset_ultra, batch_size=BATCH_SIZE_ULTRA, shuffle=True)
val_loader_ultra = DataLoader(val_dataset_ultra, batch_size=BATCH_SIZE_ULTRA)
test_loader_ultra = DataLoader(test_dataset_ultra, batch_size=BATCH_SIZE_ULTRA)

print(f"\n✅ DeBERTa-v3-large datasets created!")
print(f"   Train batches: {len(train_loader_ultra)} (x{GRAD_ACCUMULATION} accumulation = {len(train_loader_ultra)//GRAD_ACCUMULATION} effective)")
print(f"   Val batches: {len(val_loader_ultra)}")
print(f"   Test batches: {len(test_loader_ultra)} (labels are dummy for prediction)")
print(f"   Max sequence length: {MAX_LENGTH_ULTRA}")

🧹 GPU memory cleared!
   Available GPU memory: 15.83 GB

Loading DeBERTa-v3-large tokenizer...
Attempt 1: Loading from cache...
Cache not available: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached fi
Will try downloading from HuggingFace...
Attempt 2: Downloading from HuggingFace...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

✅ Tokenizer downloaded successfully!

Creating datasets...

✅ DeBERTa-v3-large datasets created!
   Train batches: 850 (x4 accumulation = 212 effective)
   Val batches: 150
   Test batches: 250 (labels are dummy for prediction)
   Max sequence length: 512


In [11]:
# 🔄 FALLBACK: Load model with multiple fallbacks (USE ONLY IF MODEL NOT LOADED)
# This cell tries multiple models in order until one succeeds

if 'model_ultra' not in dir() or model_ultra is None:
    print("Model not found in memory. Trying multiple models...")
    
    # List of models to try, in order of preference
    model_options = [
        ('roberta-large', 'RoBERTa-large', 355),
        ('roberta-base', 'RoBERTa-base', 125),
        ('distilroberta-base', 'DistilRoBERTa-base', 82),
        ('bert-base-uncased', 'BERT-base', 110),
    ]
    
    model_loaded = False
    
    for model_name, display_name, params_m in model_options:
        print(f"\n{'='*60}")
        print(f"Trying: {display_name} ({params_m}M parameters)")
        print(f"{'='*60}")
        
        try:
            # First try cache only
            print("  Attempting cache...")
            model_ultra = AutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=2,
                problem_type="single_label_classification",
                hidden_dropout_prob=DROPOUT_ULTRA,
                attention_probs_dropout_prob=DROPOUT_ULTRA,
                local_files_only=True
            )
            MODEL_NAME_ULTRA = model_name
            print(f"  ✅ {display_name} loaded from cache!")
            model_loaded = True
            break
            
        except Exception as e1:
            print(f"  Cache miss. Trying download...")
            try:
                # Try download (this might fail due to Xet Storage)
                model_ultra = AutoModelForSequenceClassification.from_pretrained(
                    model_name,
                    num_labels=2,
                    problem_type="single_label_classification",
                    hidden_dropout_prob=DROPOUT_ULTRA,
                    attention_probs_dropout_prob=DROPOUT_ULTRA,
                )
                MODEL_NAME_ULTRA = model_name
                print(f"  ✅ {display_name} downloaded successfully!")
                model_loaded = True
                break
            except Exception as e2:
                print(f"  ❌ Failed: {str(e2)[:80]}")
                print(f"  Trying next model...")
                continue
    
    if model_loaded:
        # Setup the model on GPU
        if torch.cuda.device_count() > 1:
            print(f"\n🚀 Using {torch.cuda.device_count()} GPUs with DataParallel!")
            model_ultra = torch.nn.DataParallel(model_ultra)
            device = torch.device('cuda:0')
        else:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        model_ultra = model_ultra.to(device)
        
        # Recreate optimizer and scheduler
        optimizer_ultra = AdamW(
            model_ultra.parameters(),
            lr=LEARNING_RATE_ULTRA,
            eps=1e-8,
            weight_decay=WEIGHT_DECAY_ULTRA
        )
        
        total_steps_ultra = (len(train_loader_ultra) // GRAD_ACCUMULATION) * EPOCHS_ULTRA
        warmup_steps = int(WARMUP_RATIO * total_steps_ultra)
        
        scheduler_ultra = get_linear_schedule_with_warmup(
            optimizer_ultra,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps_ultra
        )
        
        print(f"\n✅ Model initialized successfully!")
        print(f"   Using: {MODEL_NAME_ULTRA}")
        if hasattr(model_ultra, 'module'):
            params = sum(p.numel() for p in model_ultra.module.parameters())
        else:
            params = sum(p.numel() for p in model_ultra.parameters())
        print(f"   Parameters: {params:,}")
        print(f"   Device: {device}")
    else:
        print("\n❌ All model loading attempts failed!")
        print("   The HuggingFace service is down. Please:")
        print("   1. Wait 10-15 minutes and try again")
        print("   2. Or download a model to Kaggle dataset manually")
else:
    print("✅ Model already loaded in memory. Skipping reload.")

Model not found in memory. Trying multiple models...

Trying: RoBERTa-large (355M parameters)
  Attempting cache...
  Cache miss. Trying download...


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  ✅ RoBERTa-large downloaded successfully!

🚀 Using 2 GPUs with DataParallel!

✅ Model initialized successfully!
   Using: roberta-large
   Parameters: 355,361,794
   Device: cuda:0


In [12]:
# Training function with gradient accumulation and mixed precision
from torch.cuda.amp import autocast, GradScaler

def train_epoch_ultra(model, data_loader, optimizer, scheduler, device, grad_accumulation_steps=1, use_amp=True):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    
    scaler = GradScaler() if use_amp else None
    optimizer.zero_grad()
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for idx, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Mixed precision training
        with autocast(enabled=use_amp):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            if hasattr(model, 'module'):  # DataParallel
                loss = loss.mean()
            
            # Scale loss for gradient accumulation
            loss = loss / grad_accumulation_steps
        
        # Backward pass with gradient scaling
        if use_amp:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        # Update weights every grad_accumulation_steps
        if (idx + 1) % grad_accumulation_steps == 0:
            if use_amp:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * grad_accumulation_steps
        
        # Get predictions
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[:, 1]
        predictions.extend(probs.detach().cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item() * grad_accumulation_steps})
    
    avg_loss = total_loss / len(data_loader)
    auc = roc_auc_score(true_labels, predictions)
    
    return avg_loss, auc

print("✅ Training function with gradient accumulation and mixed precision ready!")

✅ Training function with gradient accumulation and mixed precision ready!


In [13]:
# Train DeBERTa-v3-large - Target 0.99!
best_auc_ultra = 0
history_ultra = {'train_loss': [], 'train_auc': [], 'val_loss': [], 'val_auc': []}
start_epoch = 0

# Optional: Resume from checkpoint
RESUME_FROM_CHECKPOINT = False  # Set to True to resume from latest checkpoint
if RESUME_FROM_CHECKPOINT:
    # Try to find the latest checkpoint
    checkpoint_files = [f for f in os.listdir('.') if f.startswith('checkpoint_deberta_ultra_epoch_')]
    if checkpoint_files:
        latest_checkpoint = sorted(checkpoint_files)[-1]
        print(f"📂 Found checkpoint: {latest_checkpoint}")
        checkpoint = load_checkpoint(model_ultra, optimizer_ultra, scheduler_ultra, latest_checkpoint)
        if checkpoint:
            start_epoch = checkpoint['epoch'] + 1
            best_auc_ultra = checkpoint['best_auc']
            history_ultra = checkpoint['history']
            print(f"✅ Resuming from epoch {start_epoch + 1}")
    else:
        print("⚠️  No checkpoints found, starting fresh")

print("=" * 80)
print("🚀 ULTRA TRAINING: DeBERTa-v3-large for 0.99 AUC")
print("=" * 80)
print(f"Multi-GPU: {torch.cuda.device_count()} GPUs")
print(f"Effective Batch Size: {BATCH_SIZE_ULTRA * GRAD_ACCUMULATION}")
print(f"Total Epochs: {EPOCHS_ULTRA}")
print(f"Starting from Epoch: {start_epoch + 1}")
print(f"Mixed Precision: Enabled (FP16)")
if RESUME_FROM_CHECKPOINT and start_epoch > 0:
    print(f"💾 Resumed from checkpoint with Best AUC: {best_auc_ultra:.4f}")
print("=" * 80)
print()

for epoch in range(start_epoch, EPOCHS_ULTRA):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch + 1}/{EPOCHS_ULTRA}")
    print(f"{'='*80}")
    
    # Train with gradient accumulation
    train_loss, train_auc = train_epoch_ultra(
        model_ultra, 
        train_loader_ultra, 
        optimizer_ultra, 
        scheduler_ultra, 
        device,
        grad_accumulation_steps=GRAD_ACCUMULATION,
        use_amp=True
    )
    history_ultra['train_loss'].append(train_loss)
    history_ultra['train_auc'].append(train_auc)
    
    # Evaluate
    val_loss, val_auc, val_preds, val_true = eval_model(model_ultra, val_loader_ultra, device)
    history_ultra['val_loss'].append(val_loss)
    history_ultra['val_auc'].append(val_auc)
    
    auc_gap = train_auc - val_auc
    
    print(f"\n📊 Results:")
    print(f"   Train Loss: {train_loss:.4f} | Train AUC: {train_auc:.4f}")
    print(f"   Val Loss:   {val_loss:.4f} | Val AUC:   {val_auc:.4f}")
    print(f"   AUC Gap:    {auc_gap:.4f}")
    
    # Save best model
    if val_auc > best_auc_ultra:
        best_auc_ultra = val_auc
        # Save the actual model (unwrap DataParallel if needed)
        model_to_save = model_ultra.module if hasattr(model_ultra, 'module') else model_ultra
        torch.save(model_to_save.state_dict(), 'best_model_deberta_v3_large_ULTRA.pt')
        print(f"   ✅ NEW BEST! Saved model with Val AUC: {best_auc_ultra:.4f}")
    
    # Delete previous checkpoint before saving new one (keep only latest)
    if epoch > start_epoch:
        old_checkpoint = f'checkpoint_deberta_ultra_epoch_{epoch}.pt'
        if os.path.exists(old_checkpoint):
            os.remove(old_checkpoint)
            print(f"   🗑️  Deleted previous checkpoint: {old_checkpoint}")
    
    # Save only the latest checkpoint for recovery
    checkpoint_path = f'checkpoint_deberta_ultra_epoch_{epoch+1}.pt'
    save_checkpoint(
        model_ultra, 
        optimizer_ultra, 
        scheduler_ultra, 
        epoch, 
        best_auc_ultra, 
        history_ultra,
        checkpoint_path
    )
    
    print()

print("\n" + "=" * 80)
print("🎉 ULTRA TRAINING COMPLETE!")
print("=" * 80)
print(f"Best Val AUC: {best_auc_ultra:.4f}")
print(f"\n📊 Comparison:")
print(f"   DistilBERT (v1-v4): Val ~0.96   → Kaggle 0.94-0.95")
print(f"   RoBERTa-base:       Val 0.9782  → Kaggle ~0.97")
print(f"   DeBERTa-v3-large:   Val {best_auc_ultra:.4f} → Expected 0.98-0.99+ 🏆")
print("=" * 80)

🚀 ULTRA TRAINING: DeBERTa-v3-large for 0.99 AUC
Multi-GPU: 2 GPUs
Effective Batch Size: 16
Total Epochs: 5
Starting from Epoch: 1
Mixed Precision: Enabled (FP16)


Epoch 1/5


Training:   0%|          | 0/850 [00:00<?, ?it/s]2025-10-20 10:20:31.128864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760955631.547852      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760955631.662869      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating: 100%|██████████| 150/150 [00:56<00:00,  2.67it/s]



📊 Results:
   Train Loss: 0.4322 | Train AUC: 0.8612
   Val Loss:   0.2372 | Val AUC:   0.9767
   AUC Gap:    -0.1155
   ✅ NEW BEST! Saved model with Val AUC: 0.9767
💾 Checkpoint saved: checkpoint_deberta_ultra_epoch_1.pt


Epoch 2/5


Evaluating: 100%|██████████| 150/150 [00:56<00:00,  2.67it/s]



📊 Results:
   Train Loss: 0.1962 | Train AUC: 0.9800
   Val Loss:   0.2184 | Val AUC:   0.9834
   AUC Gap:    -0.0034
   ✅ NEW BEST! Saved model with Val AUC: 0.9834
   🗑️  Deleted previous checkpoint: checkpoint_deberta_ultra_epoch_1.pt
💾 Checkpoint saved: checkpoint_deberta_ultra_epoch_2.pt


Epoch 3/5


...


📊 Results:
   Train Loss: 0.0739 | Train AUC: 0.9955
   Val Loss:   0.2282 | Val AUC:   0.9877
   AUC Gap:    0.0077
   ✅ NEW BEST! Saved model with Val AUC: 0.9877
   🗑️  Deleted previous checkpoint: checkpoint_deberta_ultra_epoch_2.pt
💾 Checkpoint saved: checkpoint_deberta_ultra_epoch_3.pt


Epoch 4/5


Evaluating: 100%|██████████| 150/150 [00:55<00:00,  2.69it/s]



📊 Results:
   Train Loss: 0.0271 | Train AUC: 0.9991
   Val Loss:   0.3321 | Val AUC:   0.9866
   AUC Gap:    0.0125
   🗑️  Deleted previous checkpoint: checkpoint_deberta_ultra_epoch_3.pt
💾 Checkpoint saved: checkpoint_deberta_ultra_epoch_4.pt


Epoch 5/5


Evaluating: 100%|██████████| 150/150 [00:55<00:00,  2.68it/s]



📊 Results:
   Train Loss: 0.0207 | Train AUC: 0.9993
   Val Loss:   0.2997 | Val AUC:   0.9866
   AUC Gap:    0.0127
   🗑️  Deleted previous checkpoint: checkpoint_deberta_ultra_epoch_4.pt
💾 Checkpoint saved: checkpoint_deberta_ultra_epoch_5.pt


🎉 ULTRA TRAINING COMPLETE!
Best Val AUC: 0.9877

📊 Comparison:
   DistilBERT (v1-v4): Val ~0.96   → Kaggle 0.94-0.95
   RoBERTa-base:       Val 0.9782  → Kaggle ~0.97
   DeBERTa-v3-large:   Val 0.9877 → Expected 0.98-0.99+ 🏆


In [14]:
# Load the best model and generate predictions
print("=" * 80)
print("🎯 GENERATING PREDICTIONS")
print("=" * 80)

best_model_path = 'best_model_deberta_v3_large_ULTRA.pt'

# Reinitialize model architecture
print(f"\n📥 Loading model from: {best_model_path}")
model_final = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_ULTRA,
    num_labels=2,
    problem_type="single_label_classification",
    hidden_dropout_prob=DROPOUT_ULTRA,
    attention_probs_dropout_prob=DROPOUT_ULTRA
)

# Load the best weights
model_final.load_state_dict(torch.load(best_model_path))
model_final = model_final.to(device)
model_final.eval()

print(f"✅ Model loaded successfully!")
print(f"   Model: {MODEL_NAME_ULTRA}")
if hasattr(model_final, 'module'):
    params = sum(p.numel() for p in model_final.module.parameters())
else:
    params = sum(p.numel() for p in model_final.parameters())
print(f"   Parameters: {params:,}")

# Generate predictions on test set
print(f"\n🔮 Generating predictions for {len(test_df)} test samples...")
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader_ultra, desc='Predicting'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model_final(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[:, 1]
        test_predictions.extend(probs.cpu().numpy())

print(f"\n✅ Generated {len(test_predictions)} predictions")

🎯 GENERATING PREDICTIONS

📥 Loading model from: best_model_deberta_v3_large_ULTRA.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully!
   Model: roberta-large
   Parameters: 355,361,794

🔮 Generating predictions for 1000 test samples...


Predicting: 100%|██████████| 250/250 [01:44<00:00,  2.40it/s]


✅ Generated 1000 predictions


In [15]:
# Create final submission file
print("=" * 80)
print("📝 CREATING SUBMISSION FILE")
print("=" * 80)

submission = pd.DataFrame({
    'Id': test_df['Id'],
    'TARGET': test_predictions
})

submission.to_csv('submission.csv', index=False)

print("\n✅ Submission file created: submission.csv")
print(f"   Rows: {len(submission)}")
print("\n📊 First 10 predictions:")
print(submission.head(10).to_string(index=False))

print(f"\n📊 Prediction Statistics:")
print(f"   Mean:   {submission['TARGET'].mean():.4f}")
print(f"   Median: {submission['TARGET'].median():.4f}")
print(f"   Std:    {submission['TARGET'].std():.4f}")
print(f"   Min:    {submission['TARGET'].min():.4f}")
print(f"   Max:    {submission['TARGET'].max():.4f}")

# Distribution of predictions
jailbreak_prob = (submission['TARGET'] > 0.5).sum() / len(submission)
print(f"\n📈 Prediction Distribution:")
print(f"   Predicted Jailbreak (>0.5): {(submission['TARGET'] > 0.5).sum()} ({jailbreak_prob:.2%})")
print(f"   Predicted Benign (≤0.5):    {(submission['TARGET'] <= 0.5).sum()} ({1-jailbreak_prob:.2%})")

print(f"\n🎯 SUBMISSION READY!")
print(f"   Model: {MODEL_NAME_ULTRA}")
print(f"   Expected Kaggle Score: 0.98-0.99 🏆")
if 'best_auc_ultra' in dir():
    print(f"   Best Validation AUC: {best_auc_ultra:.4f}")
print("=" * 80)

📝 CREATING SUBMISSION FILE

✅ Submission file created: submission.csv
   Rows: 1000

📊 First 10 predictions:
 Id   TARGET
  1 0.999623
  4 0.000679
  7 0.000936
 18 0.999902
 21 0.000532
 28 0.000180
 33 0.000292
 36 0.000175
 53 0.999896
 55 0.000462

📊 Prediction Statistics:
   Mean:   0.4013
   Median: 0.0012
   Std:    0.4852
   Min:    0.0001
   Max:    0.9999

📈 Prediction Distribution:
   Predicted Jailbreak (>0.5): 402 (40.20%)
   Predicted Benign (≤0.5):    598 (59.80%)

🎯 SUBMISSION READY!
   Model: roberta-large
   Expected Kaggle Score: 0.98-0.99 🏆
   Best Validation AUC: 0.9877
